**Predicting wave**

Introducción: hablar del trabajo práctico y demás..

...Primero que nada, hagamos un webscrapping para importar los datos que necesitamos

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def scrape_table(url, include_headers=True):
    # Send an HTTP request to the URL
    response = requests.get(url)

    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table in the HTML
    table = soup.find('table')

    # Initialize a list to store your data
    data = []

    # Extract the table rows
    for i, row in enumerate(table.find_all('tr')):
        # Extract the table data
        row_data = []
        for cell in row.find_all(['th', 'td']):
            row_data.append(cell.text.strip())

        # Conditionally add headers only for the first URL
        if i == 0 and include_headers:
            headers = row_data
        else:
            data.append(row_data)

    # Create a Pandas DataFrame
    if include_headers:
        df = pd.DataFrame(data, columns=headers)
    else:
        df = pd.DataFrame(data)

    return df

In [ ]:
# List of your 10 URLs
urls = ['https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_1.html',
        'https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_2.html',
        'https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_3.html',
        'https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_4.html',
        'https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_5.html',
        'https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_6.html',
        'https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_7.html',
        'https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_8.html',
        'https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_9.html',
        'https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_10.html',
        ]

In [ ]:
# Initialize an empty DataFrame to store the results
final_df = pd.DataFrame()

In [ ]:
# Scrape each URL and concatenate the results
for url in urls:
    df = scrape_table(url)
    final_df = pd.concat([final_df, df], ignore_index=True)

In [ ]:
final_df.head()

,,directorio,secuencia_p,orden,clase,dominio,mes,estrato1,sex,age,...,y_viaticos_m,y_accidentes_m,y_salarySec_m,y_ingLab_m_ha,y_gananciaNeta_m,y_gananciaNetaAgro_m,y_gananciaIndep_m,y_gananciaIndep_m_hu,y_total_m,y_total_m_ha
0,1,4514331,1,2,1,BOGOTA,1,2,0,29,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,2,4514331,1,1,1,BOGOTA,1,2,1,36,...,NA,NA,NA,8404.3203125,NA,NA,NA,NA,1620833.25,8404.3203125
2,3,4514332,1,4,1,BOGOTA,1,2,1,4,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,4,4514332,1,3,1,BOGOTA,1,2,1,7,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,5,4514332,1,1,1,BOGOTA,1,2,0,32,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [ ]:
# Mostrar el número de observaciones en el DF por control
num_observaciones = final_df.shape[0]
print("Número de observaciones:", num_observaciones)

Número de observaciones: 32177


Step 2. Trabajo de limpieza

In [ ]:

import numpy as np
# Reemplazar "NA" con NaN en la columna 'ingtot'
df_copia['y_salary_m_hu'] = df_copia['y_salary_m_hu'].replace("NA", np.nan)
# Convertir la columna 'ingtot' a tipo numérico
df_copia['y_salary_m_hu'] = pd.to_numeric(df_copia['y_salary_m_hu'], errors='coerce')
# Eliminar filas con NaN en la columna 'ingtot'
df_copia = df_copia.dropna(subset=['y_salary_m_hu'])


In [ ]:
df_copia = final_df.copy()

In [ ]:
#Elimiar individuos menores de 18 años

#Convertimos a numérica
df_copia['age'] = pd.to_numeric(df_copia['age'], errors='coerce')
# Aplicamos filtro
df_copia = df_copia[df_copia['age'] > 18]

In [ ]:
#Eliminamos observaciones con ingresos cero o sin datos
df_copia['y_salary_m_hu'] = pd.to_numeric(df_copia['y_salary_m_hu'], errors='coerce')
df_copia = df_copia[df_copia['y_salary_m_hu'] > 0]

In [ ]:
#Convertimos a logaritmo la variable de ingreso
df_copia['ingresos_log'] = np.log(df_copia['y_salary_m_hu'])

In [ ]:
categorias_ocu = df_copia['ocu'].unique()
print(categorias_ocu)

['1']


In [ ]:
df_copia = df_copia[df_copia['ocu'] == "1"]

In [ ]:
# Mostrar el número de observaciones en el DF por control
num_observaciones = df_copia.shape[0]
print("Número de observaciones:", num_observaciones)

Número de observaciones: 9785


Step 3. Análisis descriptivos

In [ ]:
## Estadísticos básicos de la variable ingresos
descripcion = df_copia['y_salary_m_hu'].describe()
print(descripcion)

count      9785.000000
mean       7984.262071
std       11629.426704
min         151.909729
25%        3797.704102
50%        4520.833496
75%        7291.666504
max      291666.656250
Name: y_salary_m_hu, dtype: float64


In [ ]:
## Media de edad de la muestra
media_edad = df_copia['age'].mean()

# Imprime la media de la edad
print(f"La media de la edad es: {media_edad:.2f}")

La media de la edad es: 36.44


In [ ]:
## % hombres en la muestra

# Convertir la columna 'sex' a tipo numérico
df_copia['sex'] = pd.to_numeric(df_copia['sex'], errors='coerce')

total_hombres = df_copia['sex'].sum()
total_muestra = len(df_copia['sex'])

if total_muestra == 0:
    print("No hay datos en la muestra.")
else:
    porcentaje_hombres = (total_hombres / total_muestra) * 100
    print(f"Porcentaje de hombres en la muestra: {porcentaje_hombres:.2f}%")


Porcentaje de hombres en la muestra: 50.18%


In [ ]:
## % ocupados formales en la muestra

# Convertir la columna 'formal' a tipo numérico
df_copia['formal'] = pd.to_numeric(df_copia['formal'], errors='coerce')

total_ocupados = df_copia['formal'].sum()
total_muestra_ocupados = len(df_copia['formal'])

if total_muestra_ocupados == 0:
    print("No hay datos en la muestra.")
else:
    porcentaje_ocupados = (total_ocupados / total_muestra) * 100
    print(f"Porcentaje de ocupados formales en la muestra: {porcentaje_ocupados:.2f}%")


Porcentaje de ocupados formales en la muestra: 77.28%


In [ ]:
categorias_p6210 = df_copia['p6210'].unique()

# Imprime las categorías únicas
print(categorias_p6210)

['5' '6' '3' '4' '1' '9']


In [ ]:
df_copia['educ_none'] = (df_copia['p6210'].astype(str).str.strip() == '1').astype(int)
df_copia['educ_primary'] = (df_copia['p6210'].astype(str).str.strip() == '3').astype(int)
df_copia['educ_secundary'] = (df_copia['p6210'].astype(str).str.strip() == '4').astype(int)
df_copia['educ_media'] = (df_copia['p6210'].astype(str).str.strip() == '5').astype(int)
df_copia['educ_univ'] = (df_copia['p6210'].astype(str).str.strip() == '6').astype(int)
df_copia['educ_nodata'] = (df_copia['p6210'].astype(str).str.strip() == '9').astype(int)

In [ ]:
prueba = df_copia['educ_nodata'].describe()
print(prueba)

count    9785.000000
mean        0.000102
std         0.010109
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: educ_nodata, dtype: float64


Step 4. Predecitve income

4.1 Primeras predicciones en muestra

In [ ]:
%matplotlib inline

# import some useful packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-whitegrid')
from statsmodels.formula.api import ols

In [ ]:
df_copia['age2'] = df_copia['age'] ** 2

In [ ]:
reg_statsmodels = ols("ingresos_log~age+age2+sex+formal+educ_primary+educ_secundary+educ_media+educ_univ+educ_nodata", data = df_copia).fit()
print(reg_statsmodels.summary())

                            OLS Regression Results                            
Dep. Variable:           ingresos_log   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                     618.9
Date:                Tue, 14 Nov 2023   Prob (F-statistic):               0.00
Time:                        00:21:37   Log-Likelihood:                -8476.9
No. Observations:                9785   AIC:                         1.697e+04
Df Residuals:                    9775   BIC:                         1.705e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          6.6104      0.103     64.

In [ ]:
#predict me simplifica hacer la operación X\hat{\beta}

# generate predicted values
yhat=reg.predict(age)

In [ ]:
# plot predicted values
lnwbar=nlsy.groupby('age')['Y'].mean()
Xbar=pd.DataFrame({'age':lnwbar.index.values})
powerlist=[Xbar['educ']**j for j in np.arange(1,10)]
Xbar=pd.concat(powerlist,axis=1)
Xbar.columns = X0.columns
ybarhat=reg.predict(Xbar)
fig = plt.figure()
ax = plt.axes()
ax.plot(Xbar['educ1'],lnwbar,'bo',Xbar['educ1'],ybarhat,'g-');
plt.title("ln Wages by Education in the NLSY")
plt.xlabel("years of schooling")
plt.ylabel("ln wages");

4.2 Predicciones fuera de muestra

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
X = df_copia[[ "age", "sex", "formal", "college",]]

Y = df_copia[["ingresos_log"]]

In [ ]:
X.describe()

,age,sex,formal,college
count,16138.000000,16138.000000,16138.000000,16138.000000
mean,39.594497,0.532842,0.598773,0.317635
std,13.348417,0.498936,0.490162,0.465571
min,19.000000,0.000000,0.000000,0.000000
25%,28.000000,0.000000,0.000000,0.000000
50%,38.000000,1.000000,1.000000,0.000000
75%,50.000000,1.000000,1.000000,1.000000
max,93.000000,1.000000,1.000000,1.000000


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        Y,
                                        test_size=0.3,
                                        train_size=0.7,
                                        random_state = 123
                                    )

In [ ]:
X_train.head()

,age,sex,formal,ocu,college
2505,65,1,0,1,0
4659,49,0,0,1,1
4423,40,1,0,1,1
26498,32,0,1,1,1
31494,56,1,1,1,0
